# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *"A non-local algorithm for image denoising"* (autorzy: Antoni Buades, Bartomeu Coll i Jean Michel Morel) na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{2}
\end{equation}

gdzie:
\begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\tag{3}
\end{equation}

- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując: jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
import time

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

mat = loadmat('MR_data.mat')
Input1 = mat['I_noisefree']
Input2 = mat['I_noisy1']
Input3 = mat['I_noisy2']
Input4 = mat['I_noisy3']
Input5 = mat['I_noisy4']


# funkcje z zadania podstawowego
def fgaussian1(size, sigma):
     m = n = size
     h, k = m // 2, n // 2
     x, y = np.mgrid[-h : h + 1, -k : k + 1]
     gauss = np.exp(-(x ** 2 + y ** 2) / (2 * sigma ** 2))
     return gauss / gauss.sum()

def fgaussian2(size, sigma):
     m = n = size
     h, k = m // 2, n // 2
     x, y = np.mgrid[-h : h + 1, -k : k + 1]
     gauss = np.exp(-(x ** 2 + y ** 2) / (2 * sigma ** 2))
     return gauss
    

def fun(window, filter, variancy):
    A, B = window.shape
    pixel = 0
    x = [A // 2, B // 2]
    for i in range(A):
        for j in range(B):
            AB = [i,j]
            y = np.sqrt(((x[0] - AB[0]) ** 2) + ((x[1] - AB[1]) ** 2))
            gauss = np.exp(-(y ** 2) / (2 * (variancy ** 2)))
            pixel = pixel + gauss * window[i,j]
    pixel = pixel / filter.sum()
    return pixel

def fun2(window, filter, variancy, r):
    A,B = window.shape
    pixel = 0
    normalization = 0
    x = [A // 2, B // 2]
    for i in range(A):
        for j in range(B):
            AB = [i,j]
            y = np.sqrt(((x[0] - AB[0]) ** 2) + ((x[1] - AB[1]) ** 2))
            gauss = np.exp(-(y ** 2) / (2 * (variancy ** 2)))
            diff = np.abs(window[A // 2, B // 2] - window[i,j])
            gauss_diff = np.exp(-(diff ** 2) / (2 * (r ** 2)))
            pixel = pixel + gauss * gauss_diff * window[i,j]
            normalization += gauss * gauss_diff
    pixel = pixel / normalization
    return pixel

def convolution(I, window, variancy):
    filter = fgaussian2(5, variancy)
    IConv = I.copy()
    X, Y = IConv.shape
    half = window // 2
    for i in range(0 + window // 2, X - window // 2):
        for j in range(0 + window // 2, Y - window // 2):
            new_window = IConv[i - half : i + half + 1, j - half : j + half + 1]
            new_pixel = fun(new_window, filter, variancy)
            IConv[i,j] = new_pixel
    return IConv

def bilateral(I, window, variancy, r):
    filter = fgaussian1(window, variancy)
    IConv = I.copy()
    X, Y = IConv.shape
    half = window // 2
    for i in range(0 + window // 2, X - window // 2):
        for j in range(0 + window // 2, Y - window // 2):
            new_window = IConv[i - half : i + half + 1, j - half : j + half + 1]
            new_pixel = fun2(new_window, filter, variancy, r)
            IConv[i,j] = new_pixel
    return IConv

In [ ]:
def NLM(I, window, area, alfa, sigma_square):
    v = fgaussian1(window, np.sqrt(sigma_square))
    IConv = I.copy()
    (X,Y)=IConv.shape
    area_half = area//2
    window_half = window//2
    for i in range(0 + area_half + window_half, X - area_half - window_half):
        for j in range(0 + area_half + window_half, Y - area_half - window_half):
                i_minus = i - area_half - window_half
                i_plus = i + area_half + window_half + 1
                j_minus = j - area_half - window_half
                j_plus = j + area_half + window_half + 1
                o = I[i_minus:i_plus,j_minus:j_plus]
                a,b = o.shape
                wpx = 0
                Zx = 0
                pixel = 0
                for k in range(0 + window_half, a - window_half):
                    for l in range(0 + window_half, b - window_half):
                        Np = o[k - window_half:k + window_half + 1, l - window_half:l + window_half + 1]
                        Nx = I[i - window_half:i + window_half + 1, j - window_half:j + window_half + 1]
                        vNp = v * Np
                        vNx = v * Nx
                        Xk = vNp - vNx
                        X_2 = np.sqrt((Xk ** 2).sum())
                        wpx = np.exp((-(X_2 ** 2)) / (alfa * sigma_square))
                        Zx = Zx + wpx
                        pixel = pixel + wpx * o[k,l]
                pixel = pixel / Zx
                IConv[i,j] = pixel
    return IConv 

In [ ]:
start_NLM = time.time()
NLM_ = NLM(Input3, 3, 7, 5, 50)
stop_NLM = time.time()
time_NLM = stop_NLM - start_NLM

start_convolution = time.time()
convolution_ = convolution(Input3, 5, 0.9)
stop_convolution = time.time()
time_convolution = stop_convolution - start_convolution

start_bilateral = time.time()
bilateral_ = bilateral(Input3, 5, 3, 45)
stop_bilateral = time.time()
time_bilateral = stop_bilateral - start_bilateral

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(25, 15))
ax1.imshow(Input3, 'gray')
ax1.set_title("Obraz oryginalny")
ax1.axis('off')
ax2.imshow(convolution_, 'gray')
ax2.set_title("Konwolucja - czas: {}".format(time_convolution))
ax2.axis('off')
ax3.imshow(bilateral_, 'gray')
ax3.set_title("Filtracja bilateralna - czas: {}".format(time_bilateral))
ax3.axis('off')
ax4.imshow(NLM_, 'gray')
ax4.set_title("Filtracja NLM - czas: {}".format(time_NLM))
ax4.axis('off')


Najlepszy wynik osiągnęła metoda NLM - najlepiej zredukowała szumy, oraz nie zamazała krawędzi. Dość dobry efekt dała także filtracja bilateralna. Najgorzej poradził sobie filtr Gaussa.
Jak widać wynik filtracji jest dość mocno powiązany z czasem wykonywania się kodu: Gauss wykonał się najszybiej, bo w około 7 sekund, a filtracja bilateralna niecałe 2 razy dłużej. Zdecydowanie najdłużej wykonywała się filtracja NLM, osiągając niemal 40 sekund.